In [93]:
from pyspark.sql import SparkSession, HiveContext
import pyspark.sql.functions as F
from pyspark.sql.types import *
import pandas as pd
import xlrd
import requests
from bs4 import BeautifulSoup
import json
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup

In [94]:
def download_content(url): 
    """Method to download the contents from webpage"""
    req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()    
    page_soup = soup(webpage, "html.parser")
    containers = page_soup.findAll("div","padded")
    for container in containers:
        hyperlinks=container.findAll('a')
        for hyperlink in hyperlinks:
            download_url=hyperlink.get('href')
            if download_url.endswith('file_name pattern'):
                download_url='url'+download_url
                print (download_url)             
                r = requests.get(download_url, stream = True)
                file_name='file_name.pdf'
                with open(file_name, 'wb') as f:
                    for chunk in r.iter_content(chunk_size = 1024*1024):
                        if chunk:
                            f.write(chunk)
                            print ("file downloaded!!")

In [96]:
url = 'url_input'
download_content(url)

In [2]:
spark = SparkSession \
    .builder \
    .appName("schema Comaprsion Df") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "10g") \
    .config("spark.executor.instances","10") \
    .enableHiveSupport()\
    .getOrCreate()

In [68]:
use_db=spark.sql('use cm2506')
show_info=spark.sql("desc cm2506.target_tbl")
show_info.registerTempTable('cmp_target_tbl')

In [69]:
df1=spark.sql("select col_name,data_type from cmp_target_tbl")
print(type(df1))

<class 'pyspark.sql.dataframe.DataFrame'>


In [91]:
input='/home/cm2506/Documents/data/input.xlsx'
loc = (input) 
  
wb = xlrd.open_workbook(loc) 
sheet = wb.sheet_by_index(0) 
sheet.cell_value(0, 0) 
  
rows='Number of rows extarcted : {0} '.format(sheet.nrows)
print(rows)
cols='Number of cols extarcted : {0} '.format(sheet.ncols)
print(cols)

Number of rows extarcted : 26 
Number of cols extarcted : 6 


In [71]:
column_list=[]
type_list=[]
for i in range(sheet.ncols): 
    col_name=sheet.cell_value(2, i)
    if col_name =='column_name':
        column_name=col_name
        for j in range(sheet.nrows):
            #print(sheet.cell_value(j, 4)) 
            if j >=3:                
                col_value=str(sheet.cell_value(j, 4)).lower()
                column_list.append(col_value)            
    if col_name == 'column_type':
        column_type=col_name
        for j in range(sheet.nrows):
            #print(sheet.cell_value(j, 5))           
            if j >=3:
                type_list.append(sheet.cell_value(j, 5))

d= {'col_name': column_list,'data_type':type_list}
#print(d)
df = pd.DataFrame(d, columns = ['col_name', 'data_type'])
print (type(df))

<class 'pandas.core.frame.DataFrame'>


In [72]:
pdf = spark.createDataFrame(df)
print(type(pdf))

<class 'pyspark.sql.dataframe.DataFrame'>


In [73]:
pdf.registerTempTable('cmp_source_tbl')
df2=spark.sql("select col_name,data_type from cmp_source_tbl")
df2.show()

+-----------------+---------+
|         col_name|data_type|
+-----------------+---------+
|         geocode |   string|
|        geopoint |   string|
|        cbsacode |   string|
| cbsadivisioncode|   string|
|cbsadivisionlevel|   string|
|cbsadivisiontitle|   string|
|       cbsalevel |   string|
|       cbsatitle |   string|
|      censusblock|   string|
|      censustract|   string|
|       countyfips|   string|
|       countyname|   string|
|       errorcode |   string|
|         latitude|   string|
|        longitude|   string|
|       placecode |   string|
|       placename |   string|
|      statuscode |   string|
|         timezone|   string|
|     timezonecode|   string|
+-----------------+---------+
only showing top 20 rows



In [87]:
cmp=spark.sql('select a.*,case when a.col_name=b.col_name then "Matched" else "Unmatched" end as matching_ind from cmp_source_tbl a join cmp_target_tbl b on a.col_name = b.col_name')

In [88]:
cmp.show()

+-----------------+---------+------------+
|         col_name|data_type|matching_ind|
+-----------------+---------+------------+
| cbsadivisioncode|   string|     Matched|
|cbsadivisionlevel|   string|     Matched|
|cbsadivisiontitle|   string|     Matched|
|      censusblock|   string|     Matched|
|      censustract|   string|     Matched|
|       countyfips|   string|     Matched|
|       countyname|   string|     Matched|
|         latitude|   string|     Matched|
|        longitude|   string|     Matched|
|         timezone|   string|     Matched|
|     timezonecode|   string|     Matched|
|         distance|   string|     Matched|
|          geohash|   string|     Matched|
+-----------------+---------+------------+



In [84]:
cmp.toPandas().to_csv('output.csv')